In [1]:
!pip install -q transformers diffusers scipy

In [10]:
import os
import cv2
import glob
import torch
import random
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm
from scipy.io import loadmat
from diffusers import MarigoldDepthPipeline
from transformers import AutoImageProcessor, AutoModelForDepthEstimation, ZoeDepthForDepthEstimation

In [11]:
IBIMS_PATH = "/kaggle/input/ibims-1/iBims-1"
ETH3D_PATH = "/kaggle/input/eth3d-dataset"

DEVICE = torch.device("cuda") if torch.cuda.is_available else torch.device("cpu")
print(DEVICE)

cuda


In [13]:
print("Scanning for depth map files...")
# Check for .pfm (Standard ETH3D depth format)
pfm_files = glob.glob(f"{ETH3D_PATH}/**/*.pfm", recursive=True)
# Check for .png (Alternative format) inside a depth folder
png_depth_files = glob.glob(f"{ETH3D_PATH}/**/*depth*/**/*.png", recursive=True)

print(f"Found {len(pfm_files)} .pfm files.")
print(f"Found {len(png_depth_files)} potential .png depth files.")

if len(pfm_files) > 0:
    print(f"Example PFM: {pfm_files[0]}")
elif len(png_depth_files) > 0:
    print(f"Example PNG: {png_depth_files[0]}")
else:
    print("WARNING: No pre-rendered depth maps found. You only have the raw 3D scans.")

Scanning for depth map files...
Found 0 .pfm files.
Found 0 potential .png depth files.


In [4]:
# https://www.asg.ed.tum.de/lmf/ibims1/

class IBimsLoader:
    def __init__(self, root_dir=IBIMS_PATH):
        self.rgb_files = sorted(glob.glob(os.path.join(root_dir, "rgb", "*.png")))
        self.depth_files = sorted(glob.glob(os.path.join(root_dir, "ibims1_core_mat", "*.mat")))
        
        if len(self.rgb_files) != len(self.depth_files):
            print("Hmm something is wrong with the dataset...")

    def __len__(self):
        return len(self.rgb_files)

    def get_item(self, idx):
        img_path = self.rgb_files[idx]
        img = cv2.imread(img_path)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        depth_path = self.depth_files[idx]
        mat_data = loadmat(depth_path)

        gt_depth = mat_data['data']['depth'][0][0]

        mask = gt_depth > 0.001
        
        return img, gt_depth, mask, os.path.basename(img_path)

class ETH3DLoader:
    def __init__(self, root_dir=ETH3D_PATH, samples_size=100, seed=42):
        self.samples = []

    def __len__(self):
        return len(self.samples)

In [5]:
#  https://huggingface.co/blog/Isayoften/monocular-depth-estimation-guide

def get_absrel(gt, pred):
    return np.mean(np.abs(gt-pred)/gt)

def get_delta(gt, pred, exponent=1):
    inlier = np.maximum((gt/pred), (pred/gt))
    return np.mean(inlier < 1.25**exponent)

def align_depth_least_square(gt_arr, pred_arr, valid_mask_arr):
    ori_shape = pred_arr.shape

    gt = gt_arr.squeeze()  # [H, W]
    pred = pred_arr.squeeze()
    valid_mask = valid_mask_arr.squeeze()
    gt_masked = gt[valid_mask].reshape((-1, 1))
    pred_masked = pred[valid_mask].reshape((-1, 1))

    # numpy solver
    _ones = np.ones_like(pred_masked)
    A = np.concatenate([pred_masked, _ones], axis=-1)
    X = np.linalg.lstsq(A, gt_masked, rcond=None)[0]
    scale, shift = X

    aligned_pred = pred_arr * scale + shift

    # restore dimensions
    aligned_pred = aligned_pred.reshape(ori_shape)

    return aligned_pred, scale, shift

In [6]:
class ModelWrapper:
    def __init__(self, choice):
        self.choice = choice
        
        if "marigold" in choice.lower():
            self.model = MarigoldDepthPipeline.from_pretrained(choice, variant="fp16").to(DEVICE)
            self.model.set_progress_bar_config(disable=True)
            print("found marigold model")
        elif "zoedepth" in choice.lower():
            self.processor = AutoImageProcessor.from_pretrained(choice)
            self.model = ZoeDepthForDepthEstimation.from_pretrained(choice).to(DEVICE)
            print("found zoedepth model")
        else:
            self.processor = AutoImageProcessor.from_pretrained(choice)
            self.model = AutoModelForDepthEstimation.from_pretrained(choice).to(DEVICE)

    def infer(self, image_path, marigold_steps=4):
        if "marigold" in self.choice.lower():
            image = Image.open(image_path).convert("RGB")
            w0, h0 = image.size
            
            with torch.no_grad():
                pipe_out = self.model(
                    image, 
                    num_inference_steps=marigold_steps, 
                    output_type="pt"
                )
            prediction = pipe_out.prediction

            # Check if resize is needed
            if prediction.shape[-2:] != (h0, w0):
                prediction = torch.nn.functional.interpolate(
                    prediction,
                    size=(h0, w0),
                    mode="bicubic",
                    align_corners=False
                )

            return prediction.squeeze().cpu().numpy()

        # Transformer-based models inference
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        inputs = self.processor(images=image, return_tensors="pt").to(DEVICE)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
            predicted_depth = outputs.predicted_depth
        
        # Resize to original image size
        prediction = torch.nn.functional.interpolate(
            predicted_depth.unsqueeze(1),
            size=image.shape[:2],
            mode="bicubic",
            align_corners=False,
        )
        
        return prediction.squeeze().cpu().numpy()

In [75]:
# Check if dataset gt is in mm
for i in range(len(dataset)):
    _, gt, _, _ = dataset.get_item(i)
    if np.median(gt) > 100:
        print("maybe that measurement is in millimeters... needs to change!")

In [7]:
ibims = IBimsLoader()

def benchmark(model, dataset, relative=True):
    absrel_list = []
    delta_list = []
    
    for i in tqdm(range(len(dataset))):
        img, gt, mask, name = dataset.get_item(i)
        image_path = f"{IBIMS_PATH}/rgb/{name}"
            
        prediction = model.infer(image_path)
        if relative:
            depth_trans, _, _ = align_depth_least_square(gt, prediction, mask)
            absrel = get_absrel(gt[mask], depth_trans[mask])
            delta = get_delta(gt[mask], depth_trans[mask], 1)
        else:
            absrel = get_absrel(gt[mask], prediction[mask])
            delta = get_delta(gt[mask], prediction[mask], 1)
    
        absrel_list.append(absrel)
        delta_list.append(delta)
    
    print(f"Average Abs Rel: {np.mean(absrel_list)}")
    print(f"Average d_1: {np.mean(delta_list)}")

In [8]:
benchmark(ModelWrapper("depth-anything/Depth-Anything-V2-Large-hf"), ibims)

preprocessor_config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

100%|██████████| 100/100 [00:42<00:00,  2.34it/s]

Average Abs Rel: 0.1252612876460376
Average d_1: 0.8572297850934841


In [9]:
benchmark(ModelWrapper("depth-anything/Depth-Anything-V2-Metric-Indoor-Large-hf"), ibims, relative=False)

preprocessor_config.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

100%|██████████| 100/100 [00:38<00:00,  2.60it/s]

Average Abs Rel: 0.1264849104849234
Average d_1: 0.886137028246952


In [17]:
benchmark(ModelWrapper("Intel/zoedepth-nyu-kitti"), ibims, relative=False)

found zoedepth model


100%|██████████| 100/100 [00:36<00:00,  2.71it/s]

Average Abs Rel: 0.20092700231619312
Average d_1: 0.6119498442326409


In [18]:
# MiDaS 3.0
benchmark(ModelWrapper("Intel/dpt-large"), ibims)

preprocessor_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/942 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 100/100 [00:18<00:00,  5.44it/s]

Average Abs Rel: 0.13541643258733274
Average d_1: 0.8394842777713843


In [19]:
# MiDaS 3.1
benchmark(ModelWrapper("Intel/dpt-beit-large-512"), ibims)

preprocessor_config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.38G [00:00<?, ?B/s]

100%|██████████| 100/100 [00:55<00:00,  1.82it/s]

Average Abs Rel: 0.12628050729642812
Average d_1: 0.8533818513122847


In [8]:
# Marigold with 4 iterations
benchmark(ModelWrapper("prs-eth/marigold-depth-v1-1"), ibims)

model_index.json:   0%|          | 0.00/534 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

...

vae/diffusion_pytorch_model.fp16.safeten(…):   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

found marigold model


100%|██████████| 100/100 [02:43<00:00,  1.63s/it]

Average Abs Rel: 0.053644879019413894
Average d_1: 0.9681597840673872
